In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\Next_Word_Predictor\\research'

In [3]:
os.chdir("../")

In [4]:

%pwd

'c:\\DataScience\\Projects\\Next_Word_Predictor'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    array_path: Path
    metric_file_name: Path

In [6]:
from NWPproject.constants import *
from NWPproject.utils.common import read_yaml_file, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path = config.model_path,
            array_path=config.array_path,
            metric_file_name = config.metric_file_name,           
        )

        return model_evaluation_config

In [8]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
import numpy as np
import joblib
from NWPproject.utils.common import save_json_file

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, model, X, y, Y):
        loss = sparse_categorical_crossentropy(Y,model.predict(X))
        perplexity = np.exp(np.mean(loss))

        _, accuracy = model.evaluate(X, y, verbose=0)

        return perplexity, accuracy

    
    def save_results(self):

        model = joblib.load(self.config.model_path)

        loaded_arrays = np.load(self.config.array_path)

        loaded_array1 = loaded_arrays['arr1']
        loaded_array2 = loaded_arrays['arr2']
        loaded_array3 = loaded_arrays['arr3']
        

        (perplexity, accuracy) = self.eval_metrics(model,loaded_array1,loaded_array2, loaded_array3)
        
        # Saving metrics as local
        scores = {"perplexity": float(perplexity), "accuracy": float(accuracy)}
        print(scores)
        print(type(scores))
        save_json_file(file_path=Path(self.config.metric_file_name), data=scores)

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-07-03 21:50:48,910: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-07-03 21:50:48,921: INFO:common: yaml file: params.yaml loaded successfully]
[2024-07-03 21:50:48,927: INFO:common: createD Directory at:artifacts]
[2024-07-03 21:50:48,927: INFO:common: createD Directory at:artifacts/model_evaluation]
21/21 [==============================] - 1s 8ms/step
{'perplexity': 1.3306827545166016, 'accuracy': 0.961309552192688}
<class 'dict'>
[2024-07-03 21:50:50,804: INFO:common: Data has been saved to artifacts\model_evaluation\metrcis.json]


In [11]:
from pathlib import Path